# Table of Contents

 1. Creating longitude-latitude table for later reference
 2. Scaling values for analysis and visualization
 3. Creating world ranking data
 4. Creating USA-specific data - scaled version
 5. Creating USA-specific data - ranking version


#### result data:
 - lat_long.csv
 - quality_of_life_index_5yrs_original_geo.csv
 - quality_of_life_index_5yrs_scale_geo.csv
 - quality_of_life_index_5yrs_ranking_world.csv
 - quality_of_life_index_5yrs_scale_geo_USA.csv
 - quality_of_life_index_5yrs_ranking_USA.csv

# 1. Creating longitude-latitude table for later reference

In [1]:
import pandas as pd

In [2]:
# Importing country_city_list.csv

list_city_country_continent = pd.read_csv('../02_data/city_country_continent_list.csv')
list_city_country_continent.head()

,City,Country,Continent
0,Tucson,United States,America
1,San Jose,Costa Rica,America
2,Miami,United States,America
3,Nizhny Novgorod,Russia,Europe
4,Rotterdam,Netherlands,Europe


In [13]:
## split America into North America and Latin America

# North America
list_city_country_continent.loc[(list_city_country_continent.Country == 'United States'
                                ), 'Continent'] = 'North America'
list_city_country_continent.loc[(list_city_country_continent.Country == 'Canada'
                                ), 'Continent'] = 'North America'
# Latin America
list_city_country_continent.loc[(list_city_country_continent.Continent == 'America'
                                ), 'Continent'] = 'Latin America'

In [14]:
## importing the package used to obtain the latitude and longitude of each city 

import geopandas as gpd
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")

# Initializing the lists used to store the latitudes and longitudes

city = []
country = []
latitudes = []
longitudes = []
continent = []

# Obtaining and store the latitude and longitude of each city

for row in range(len(list_city_country_continent.index)):
    location = geolocator.geocode(
        list_city_country_continent['City'][row] + ', ' + list_city_country_continent['Country'][row]
    )
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)
    city.append(list_city_country_continent['City'][row])
    country.append(list_city_country_continent['Country'][row])
    continent.append(list_city_country_continent['Continent'][row])

lat_long = pd.DataFrame({'City': city, 'Country': country, 'Continent' : continent, 
                         'Latitude': latitudes, 'Longitude': longitudes})

# exporting result dataframe to csv

lat_long.to_csv('../04_data/lat_long.csv', index = False)


######### Takes some time... #########

In [15]:
lat_long

,City,Country,Continent,Latitude,Longitude
0,Tucson,United States,North America,32.222876,-110.974848
1,San Jose,Costa Rica,Lain America,9.932543,-84.079578
2,Miami,United States,North America,25.774173,-80.193620
3,Nizhny Novgorod,Russia,Europe,56.326482,44.005139
4,Rotterdam,Netherlands,Europe,51.924442,4.477750
...,...,...,...,...,...
255,Quito,Ecuador,Lain America,-0.220164,-78.512327
256,Kuala Lumpur,Malaysia,Asia,3.151696,101.694237
257,Budapest,Hungary,Europe,47.497994,19.040359
258,Cairo,Egypt,Africa,30.044388,31.235726


In [16]:
## adding latitude and longitude values to the original data

# importing original data

qol_22 = pd.read_csv('../02_data/quality_of_life_index_2022.csv')
qol_21 = pd.read_csv('../02_data/quality_of_life_index_2021.csv')
qol_20 = pd.read_csv('../02_data/quality_of_life_index_2020.csv')
qol_19 = pd.read_csv('../02_data/quality_of_life_index_2019.csv')
qol_18 = pd.read_csv('../02_data/quality_of_life_index_2018.csv')
lat_long = pd.read_csv('../04_data/lat_long.csv')

# matching city-country and add latitude/longitude and continent

qol_22_geo = qol_22.merge(lat_long, how='inner', 
                          left_on=['City', 'Country'], right_on=['City', 'Country'])
qol_21_geo = qol_21.merge(lat_long, how='inner', 
                          left_on=['City', 'Country'], right_on=['City', 'Country'])
qol_20_geo = qol_20.merge(lat_long, how='inner', 
                          left_on=['City', 'Country'], right_on=['City', 'Country'])
qol_19_geo = qol_19.merge(lat_long, how='inner', 
                          left_on=['City', 'Country'], right_on=['City', 'Country'])
qol_18_geo = qol_18.merge(lat_long, how='inner', 
                          left_on=['City', 'Country'], right_on=['City', 'Country'])

In [17]:
# merging yearly dataframe across 2018-2022

qol_5yrs_geo = pd.concat([qol_22_geo, qol_21_geo, qol_20_geo, qol_19_geo, qol_18_geo], 
                         axis=0, join='outer', ignore_index=True)

In [8]:
# exporting result dataframe to csv

qol_5yrs_geo.to_csv('../04_data/quality_of_life_index_5yrs_original_geo.csv', index = False)

# 2. Scaling values for analysis and visualization

In [9]:
# importing 2018 ~ 2022 data   
qol_22 = pd.read_csv('../02_data/quality_of_life_index_2022.csv')
qol_21 = pd.read_csv('../02_data/quality_of_life_index_2021.csv')
qol_20 = pd.read_csv('../02_data/quality_of_life_index_2020.csv')
qol_19 = pd.read_csv('../02_data/quality_of_life_index_2019.csv')
qol_18 = pd.read_csv('../02_data/quality_of_life_index_2018.csv')

In [10]:
## defining functions for scaling indexes through min-max normalization with range [0,100] 

def min_max_scaling(series):
    return (series - series.min())*100/(series.max()-series.min())

def min_max_scaling_inverse(series):
    return 100-((series - series.min())*100/(series.max()-series.min()))

def scale(qol_22):
    qol_22['Quality of Life Index'] = min_max_scaling(qol_22['Quality of Life Index'])
    qol_22['Purchasing Power Index'] = min_max_scaling(qol_22['Purchasing Power Index'])
    qol_22['Safety Index'] = min_max_scaling(qol_22['Safety Index'])
    qol_22['Healthcare Index'] = min_max_scaling(qol_22['Healthcare Index'])
    qol_22['Cost of Living Index'] = min_max_scaling_inverse(qol_22['Cost of Living Index'])
    qol_22['Property Price to Income Ratio'] = min_max_scaling_inverse(qol_22['Property Price to Income Ratio'])
    qol_22['Traffic Commute Time Index'] = min_max_scaling_inverse(qol_22['Traffic Commute Time Index'])
    qol_22['Pollution Index'] = min_max_scaling_inverse(qol_22['Pollution Index'])
    qol_22['Climate Index'] = min_max_scaling(qol_22['Climate Index'])

In [11]:
## scale(min-max normalization) yearly data by looping

list_ = [qol_22, qol_21, qol_20, qol_19, qol_18]
for df in list_:
    scale(df)

In [13]:
## merging yearly scaled data across 2018~2022

qol_5yrs_scale = pd.concat([qol_22,qol_21, qol_20, qol_19, qol_18], axis=0, 
                           join='outer', ignore_index=True,)

qol_5yrs_scale.head(10)

,City,Country,Quality of Life Index,Purchasing Power Index,Safety Index,Healthcare Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index,Year
0,Raleigh,United States,100.000000,82.475103,68.775031,75.978793,5.705494e+01,96.733569,67.413725,89.230209,81.260946,2022
1,Canberra,Australia,99.775281,58.786738,85.347328,89.294454,5.004978e+01,92.168438,85.025783,99.562895,79.906597,2022
2,Adelaide,Australia,98.232615,58.258441,75.928739,86.398858,4.946149e+01,94.352617,84.490282,93.734823,94.197315,2022
3,Columbus,United States,96.125114,86.829002,56.414860,72.532626,5.569735e+01,94.864227,82.982943,84.276348,66.561588,2022
4,Zurich,Switzerland,95.815366,73.773379,93.205358,73.062806,1.421085e-14,87.111373,67.096390,94.038368,78.458844,2022
5,Madison,United States,94.776799,75.194316,69.424113,80.342577,5.785139e+01,95.808737,84.490282,92.751336,43.619381,2022
6,Austin,United States,94.612815,91.031091,65.198177,66.598695,5.859354e+01,95.789059,64.597382,71.478873,79.159370,2022
7,Charlotte,United States,93.659277,82.936604,53.818533,73.919250,5.537153e+01,97.796143,63.566045,81.265177,81.459428,2022
8,Oklahoma City,United States,92.936532,72.656060,48.280624,76.264274,6.000543e+01,97.068083,82.863943,89.060223,73.940455,2022
9,Basel,Switzerland,92.717886,62.685208,88.095567,81.933116,2.805684e-01,86.265250,91.213804,87.117533,80.023351,2022


In [17]:
# add latitude/longitude and continent by matching city-country

qol_5yrs_scale_geo = qol_5yrs_scale.merge(lat_long, how='inner', 
                                          left_on=['City', 'Country'], right_on=['City', 'Country'])

In [18]:
# exporting merged dataframe to csv

qol_5yrs_scale_geo.to_csv('../04_data/quality_of_life_index_5yrs_scale_geo.csv', index = False)

# 3. Creating world ranking data

In [19]:
# importing data

qol_scale = pd.read_csv('../04_data/quality_of_life_index_5yrs_scale_geo.csv')

qol_scale_22 = qol_scale.loc[:][qol_scale['Year'] == 2022]
qol_scale_21 = qol_scale.loc[:][qol_scale['Year'] == 2021]
qol_scale_20 = qol_scale.loc[:][qol_scale['Year'] == 2020]
qol_scale_19 = qol_scale.loc[:][qol_scale['Year'] == 2019]
qol_scale_18 = qol_scale.loc[:][qol_scale['Year'] == 2018]

# listing up columns for ranking

rankcols = ['Quality of Life Index',
       'Purchasing Power Index', 'Safety Index', 'Healthcare Index',
       'Cost of Living Index', 'Property Price to Income Ratio',
       'Traffic Commute Time Index', 'Pollution Index', 'Climate Index']

# converting values to ranking values 

qol_scale_22[rankcols] = qol_scale_22[rankcols].rank('rows', ascending=False).astype(int)
qol_scale_21[rankcols] = qol_scale_21[rankcols].rank('rows', ascending=False).astype(int)
qol_scale_20[rankcols] = qol_scale_20[rankcols].rank('rows', ascending=False).astype(int)
qol_scale_19[rankcols] = qol_scale_19[rankcols].rank('rows', ascending=False).astype(int)
qol_scale_18[rankcols] = qol_scale_18[rankcols].rank('rows', ascending=False).astype(int)

In [20]:
# merging

qol_5yrs_world_ranking = pd.concat([qol_scale_22, qol_scale_21, qol_scale_20, 
                                    qol_scale_19, qol_scale_18], axis=0, join='outer', 
                                   ignore_index=True,)
qol_5yrs_world_ranking

,City,Country,Quality of Life Index,Purchasing Power Index,Safety Index,Healthcare Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index,Year,Continent,Latitude,Longitude
0,Raleigh,United States,1,9,79,51,152,22,105,16,109,2022,America,35.780398,-78.639099
1,Canberra,Australia,2,54,18,7,204,57,13,2,118,2022,Oceania,-35.297591,149.101268
2,Adelaide,Australia,3,56,52,13,207,46,15,9,41,2022,Oceania,-34.928181,138.599931
3,Columbus,United States,4,5,127,75,165,40,17,36,175,2022,America,39.962260,-83.000707
4,Zurich,Switzerland,5,20,7,72,255,96,109,8,128,2022,Europe,47.374449,8.541042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,Colombo,Sri Lanka,167,180,64,29,29,178,182,113,158,2018,Asia,6.938747,79.854113
1139,Dhaka,Bangladesh,182,168,176,182,19,135,179,181,126,2018,Asia,23.786198,90.402615
1140,Tehran,Iran,172,157,141,171,30,132,175,172,132,2018,Asia,35.689252,51.389600
1141,Manila,Philippines,179,158,157,103,44,153,181,177,154,2018,Asia,14.594891,120.978262


In [21]:
# exporting merged data to csv

qol_5yrs_world_ranking.to_csv('../04_data/quality_of_life_index_5yrs_ranking_world.csv', index = False)

# 4. Creating USA-specific data - scaled version

In [23]:
## Creating scaled data for USA cities only

# importing original data
qol = pd.read_csv('../04_data/quality_of_life_index_5yrs_original_geo.csv')

qol_22 = qol.loc[:][qol['Year'] == 2022]
qol_21 = qol.loc[:][qol['Year'] == 2021]
qol_20 = qol.loc[:][qol['Year'] == 2020]
qol_19 = qol.loc[:][qol['Year'] == 2019]
qol_18 = qol.loc[:][qol['Year'] == 2018]

# pick USA cities 

qol_22_us = qol_22[:].loc[(qol_22['Country'] == 'United States')]
qol_21_us = qol_21[:].loc[(qol_21['Country'] == 'United States')]
qol_20_us = qol_20[:].loc[(qol_20['Country'] == 'United States')]
qol_19_us = qol_19[:].loc[(qol_19['Country'] == 'United States')]
qol_18_us = qol_18[:].loc[(qol_18['Country'] == 'United States')]

In [24]:
# scaling (min-max normalization with range [0,100])

list_ = [qol_22_us, qol_21_us, qol_20_us, qol_19_us, qol_18_us]

for df in list_:
    scale(df)

In [25]:
# merge yearly data

qol_5yrs_scale_us = pd.concat([qol_22_us, qol_21_us, qol_20_us, qol_19_us, qol_18_us], 
                              axis=0, join='outer', ignore_index=True,)
qol_5yrs_scale_us

,City,Country,Quality of Life Index,Purchasing Power Index,Safety Index,Healthcare Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index,Year,Continent,Latitude,Longitude
0,Raleigh,United States,100.000000,68.057554,95.842192,77.662957,86.337068,80.584795,72.308872,88.947266,75.956873,2022,America,35.780398,-78.639099
1,Columbus,United States,91.470588,75.993359,74.698795,70.945946,82.683877,69.473684,92.380465,81.007978,53.333333,2022,America,39.962260,-83.000707
2,Madison,United States,88.502674,54.786940,96.952516,86.168521,88.480273,75.087719,94.323702,94.590387,18.023360,2022,America,43.074761,-89.383761
3,Austin,United States,88.141711,83.652463,89.723600,59.379968,90.477350,74.970760,68.678087,60.498151,72.722372,2022,America,30.271129,-97.743700
4,Charlotte,United States,86.042781,68.898727,70.257501,73.648649,81.807112,86.900585,67.348504,76.182137,76.262354,2022,America,35.227209,-80.843083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,Chicago,United States,21.818702,37.761130,13.401150,31.755964,50.767656,76.460177,9.708738,57.361683,38.468271,2018,America,41.875562,-87.624421
215,Miami,United States,21.804404,6.987466,38.124724,28.783731,35.798362,58.230088,16.504854,55.921353,74.857768,2018,America,25.774173,-80.193620
216,Detroit,United States,10.766371,55.698026,0.000000,0.000000,76.253838,100.000000,0.000000,27.023320,37.592998,2018,America,42.331551,-83.046640
217,Los Angeles,United States,9.965685,23.123469,50.862450,29.487681,45.291709,34.690265,0.000000,4.641061,97.242888,2018,America,34.053691,-118.242766


In [26]:
# exporting merged dataframe to csv

qol_5yrs_scale_us.to_csv('../04_data/quality_of_life_index_5yrs_scale_geo_USA.csv', index = False)

# 5. Creating USA-specific data - ranking version

In [27]:
# importing scaled data

qol_5yrs_scale_geo_USA = pd.read_csv('../04_data/quality_of_life_index_5yrs_scale_geo_USA.csv')

In [28]:
# dividing the data according to year

qol_22_us = qol_5yrs_scale_geo_USA[:].loc[(qol_5yrs_scale_geo_USA['Year'] == 2022)]
qol_21_us = qol_5yrs_scale_geo_USA[:].loc[(qol_5yrs_scale_geo_USA['Year'] == 2021)]
qol_20_us = qol_5yrs_scale_geo_USA[:].loc[(qol_5yrs_scale_geo_USA['Year'] == 2020)]
qol_19_us = qol_5yrs_scale_geo_USA[:].loc[(qol_5yrs_scale_geo_USA['Year'] == 2019)]
qol_18_us = qol_5yrs_scale_geo_USA[:].loc[(qol_5yrs_scale_geo_USA['Year'] == 2018)]

In [29]:
# listing up columns for ranking

rankcols = ['Quality of Life Index',
       'Purchasing Power Index', 'Safety Index', 'Healthcare Index',
       'Cost of Living Index', 'Property Price to Income Ratio',
       'Traffic Commute Time Index', 'Pollution Index', 'Climate Index']

# converting values to ranking values 

qol_22_us[rankcols] = qol_22_us[rankcols].rank('rows', ascending=False).astype(int)
qol_21_us[rankcols] = qol_21_us[rankcols].rank('rows', ascending=False).astype(int)
qol_20_us[rankcols] = qol_20_us[rankcols].rank('rows', ascending=False).astype(int)
qol_19_us[rankcols] = qol_19_us[rankcols].rank('rows', ascending=False).astype(int)
qol_18_us[rankcols] = qol_18_us[rankcols].rank('rows', ascending=False).astype(int)

In [30]:
# merging

qol_5yrs_us_ranking = pd.concat([qol_22_us, qol_21_us, qol_20_us, qol_19_us, qol_18_us], 
                                axis=0, join='outer', ignore_index=True,)

In [31]:
# exporting merged data to csv

qol_5yrs_us_ranking.to_csv('../04_data/quality_of_life_index_5yrs_ranking_USA.csv', 
                           index = False)